# Load packages and configuration

In [11]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from equilibrator_api import ComponentContribution, Q_ #for creating quantity variables
import equilibrator_api
import equilibrator_custom_functions_paper as eq #custom functions created during analysis
import numpy as np #numerical package
import math
import matplotlib.pyplot as plt #plotting package
import custom_plot_functions as cpf

import pandas as pd

# if more space is required
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>div.output_scroll {height: 100em; }</style>"))

# set defaults for plotting - grid on, linewidth = 2
plt.rc( 'axes', grid=True  )
plt.rc( 'figure', figsize = (7,4), dpi=96)
plt.rc( 'axes', linewidth=1 )
plt.rc( 'lines', linewidth=2 )

# Define physiological conditions and metabolite ranges

In [2]:
#E. coli physiological conditions as a dictionary
E_coli_con = {
            "p_h": str(Q_("7.5")),
            "p_mg": str(Q_(6)),
            "ionic_strength": str(Q_('200 mM')),
            "temperature": str(Q_(37+273.15,"K")),
            "kcat_source": "fwd",
}
cs_default_bounds = eq.obtain_compound_settings("metabolite_reference_table_expanded_custom_phys_conc_paper", custom_bounds = True)

cc= ComponentContribution()

# Compute MDFs for all pathway combinations


In [9]:
%%time

results_excel= 'Filename'
xls = pd.ExcelFile(results_excel+'.xlsx')
sheets = xls.sheet_names[1:]
xls.close()

mdf_simulations_all={}

for sheet in sheets:
    print(sheet)
    # Perform MDF Simulation
    MDF_result = eq.MDF_analysis(results_excel,sheet,cs_default_bounds,E_coli_con, y=0, custom_dGs = False, print_results = True)
    MDF,res1,g1,g2,g3,g4,flx,shp_r,r_id,res2,met,bl,bu,c,shp_c = eq.print_MDF_results_individual(MDF_result)

    # Save results in dictionary
    mdf_simulations_all[sheet] = [MDF,MDF_result.reaction_df, MDF_result.compound_df, res1,g1,g2,g3,g4,flx,shp_r,r_id,res2,met,bl,bu,c,shp_c] 
    

2__EM+GAA Pathway
0.375 O2 + Methanol <=> 1.125 H2O + 0.125 3-hydroxybutanol-butanoate


'MDF = 5.60 kJ/mol'

4__NOG+RuMP Cycle (TAL)
0.375 O2 + Methanol <=> 1.125 H2O + 0.125 3-hydroxybutanol-butanoate


'MDF = 4.76 kJ/mol'

5__ED+Homoserine Pathway
0.375 O2 + 1e-06 NADP + 1e-06 NADH + Methanol <=> 1.125 H2O + 1e-06 NADPH + 1e-06 NAD + 0.125 3-hydroxybutanol-butanoate


...

17__NOG+GAPA Pathway
0.375 O2 + Methanol <=> 1.125 H2O + 0.125 3-hydroxybutanol-butanoate


'MDF = 5.60 kJ/mol'

18__ED+SACA Pathway
0.375 O2 + Methanol <=> 1.125 H2O + 0.125 3-hydroxybutanol-butanoate


'MDF = 5.60 kJ/mol'

CPU times: total: 7min 41s
Wall time: 9min 6s


## Save results in Excel

In [10]:
# Store all MDF scores in an Excel file 

MDF_score_dict={}

for sheet_MDF in sheets:
    MDF_score_dict[sheet_MDF]=np.round(mdf_simulations_all[sheet_MDF][0],3)
    
MDF_score_df=pd.DataFrame.from_dict(MDF_score_dict, orient='index')
MDF_score_df.columns=['MDF Score (kJ/mol)']

with pd.ExcelWriter("PHB_MeOH_EnzCoup_MDF.xlsx", mode="w", engine="openpyxl") as writer:
    MDF_score_df.to_excel(writer, sheet_name='Summary', encoding='UTF-8')

    
# Store all data and graphs from each MDF calculation in an independent sheet
import openpyxl

for sheet_MDF_data in sheets:
    with pd.ExcelWriter("PHB_MeOH_EnzCoup_MDF.xlsx", mode="a", engine="openpyxl", if_sheet_exists ='overlay') as writer:
        
        # Save reaction information
        react_df= mdf_simulations_all[sheet_MDF_data][1]
        react_df.to_excel(writer,sheet_name=sheet_MDF_data,startrow=0 , startcol=0)   
        
        # Save metabolite information
        metab_df= mdf_simulations_all[sheet_MDF_data][2]
        metab_df.to_excel(writer,sheet_name=sheet_MDF_data,startrow=(len(react_df))+3 , startcol=0)   

## Create plot with all simulations

In [ ]:
# Create plot with all simulations

import math 
# integrate in one figure
fig, axs = plt.subplots(math.ceil((len(mdf_simulations_all.keys()))/3),3, figsize=(30,40), facecolor="white" )

ind = 0

for mdf_sim in mdf_simulations_all.keys():
    
    # Extract all data necessary for the plot from the raw simulation data
    MDF = mdf_simulations_all[mdf_sim][0]
    g3 = mdf_simulations_all[mdf_sim][6]
    g4 = mdf_simulations_all[mdf_sim][7]
    r_id = mdf_simulations_all[mdf_sim][10]
    shp_r = mdf_simulations_all[mdf_sim][9]

    # Set the position of the graph for every for loop
    vert_indx= math.floor(ind/3)
    lat_indx= ind -3*vert_indx
    
    # Plot and edit the graph
    cpf.plot_bar_chart( MDF,g3,g4,r_id,shp_r, ax=axs[vert_indx,lat_indx] )
    axs[vert_indx,lat_indx].set_title(mdf_sim +' = '+ str(np.round(MDF,3)) + ' kJ/mol', fontsize=10)
    axs[vert_indx,lat_indx].set_ylim( [-60, 40] )
    axs[vert_indx,lat_indx].tick_params( axis='x', rotation=90 )
    axs[vert_indx,lat_indx].legend(loc=3)
    ind = ind+1
        #cpf.plot_concentrations_my(MDF_result)  
        #display(res1)
        #display(res2)

fig.tight_layout()
fig.show()
        
        



## Create plots for individual simulations

In [ ]:
from importlib import reload
reload(cpf)

import openpyxl

wb = openpyxl.load_workbook("BAME_Xyl&MeOH_MDF_Final.xlsx")        

for sheet_MDF_data in wb.sheetnames[1:]:
        # Create and save MDF graph
        
        # Extract all data necessary for the plot from the raw simulation data
        MDF = mdf_simulations_all[sheet_MDF_data][0]
        g3 = mdf_simulations_all[sheet_MDF_data][6]
        g4 = mdf_simulations_all[sheet_MDF_data][7]
        r_id = mdf_simulations_all[sheet_MDF_data][10]
        shp_r = mdf_simulations_all[sheet_MDF_data][9]
 
        # Plot and edit the graph
        cpf.plot_bar_chart( sheet_MDF_data,MDF,g3,g4,r_id,shp_r)
                
        # Save graph
        graph_name= sheet_MDF_data + ' MDF Reaction Plot.png' 
        plt.savefig(graph_name, bbox_inches='tight')
        
        
        #writer = pd.ExcelWriter('PHB_Glc_MDF.xlsx', mode='a', engine = 'openpyxl', if_sheet_exists ='overlay' )
        
        
        wb = openpyxl.load_workbook("BAME_Xyl&MeOH_MDF_Final.xlsx")
        ws = wb[sheet_MDF_data]
        print(ws)
        img = openpyxl.drawing.image.Image(graph_name)
        #display(img)
        img.anchor = 'L2' # Or whatever cell location you want to use.
        ws.add_image(img)
        #wb.save("PHB_Glc_MDF_with_image.xlsx")
        
        
        #graph1 = openpyxl.drawing.image.Image(graph_name)
        
        
        
        #display(graph1)
        #ws.add_image(graph, 'B3')
        #wb.save('test.xlsx')
        
        #worksheet = writer.sheets[sheet_MDF_data]
        #worksheet.insert_image('L1',graph_name)
        #writer.save()
        #MDF_result.reaction_df

# Sensitivity Analysis

In [ ]:
%%time

import equilibrator_custom_functions_paper as eq #custom functions created during analysis
from imp import reload
reload(eq)

In [ ]:


E_coli_con = {
            "p_h": str(Q_("7.5")),
            "p_mg": str(Q_(0)),
            "ionic_strength": str(Q_('200 mM')),
            "temperature": str(Q_(37+273.15,"K")),
            "kcat_source": "fwd",
}

results_excel= 'Noreugenin_v5_Glc_for_MDF'
xls = pd.ExcelFile(results_excel+'.xlsx')
sheets = xls.sheet_names[1:]
xls.close()

var_simulations_all={}

for sheet_CVA in sheets:
    print(sheet_CVA)
    # Perform CVA Simulation
    CVA_result= eq.MDF_CVA(0,results_excel,sheet_CVA,cs_default_bounds,E_coli_con, compounds= None, custom_dGs = False, print_results = True, y = 0)
    var_simulations_all[sheet_CVA]=pd.DataFrame(CVA_result, columns= ['Metabolite', 'Lower bound', 'Upper bound'])
    break

In [ ]:
for sheet_CVA_data in sheets:
     with pd.ExcelWriter("Noreugenin_v5_Glc.xlsx", mode="w", engine="openpyxl") as writer:
        
        # Save reaction information
        cva_df= var_simulations_all[sheet_CVA_data]
        cva_df.to_excel(writer,sheet_name=sheet_cva_data,startrow=0 , startcol=0)

In [ ]:
display(var_simulations_all['1__NOG+Homoserine Pathway'])

In [ ]:
type(CVA_result)

In [ ]:
print(CVA_result)

In [ ]:
print(8.3145*310.15)